In [58]:
#for building the pyomo model
import pyomo.environ as pe

In [59]:
#for solving the pyomo model
import pyomo.opt as po

#### Three Product Problem 

Cristina Hernández, Macarena Vargas, Guillermo Ruiz, Beatriz Jiménez. 3ºIMAT B 

### Create the model

In [60]:
model = pe.ConcreteModel()

#### Sets
$i$: products {'A', 'B', 'C'}

$j$: factories {'F1', 'F2', 'F3', 'F4', 'F5', 'F6'}

In [61]:
model.product = pe.Set(initialize=['A', 'B', 'C'])
model.factory = pe.Set(initialize=['F1', 'F2', 'F3', 'F4', 'F5', 'F6'])

#### Parameters
$C_{ij}$: unitary cost of product $i$ in factory $j$ [€/unit] 


In [62]:
#unitary cost of product in each factory
product_costs_dict = {
    ('A', 'F1'): 25, ('A', 'F2'): 30, ('A', 'F3'): 26, ('A', 'F4'): 34, ('A', 'F5'): 32, ('A', 'F6'): 30,
    ('B', 'F1'): 30, ('B', 'F2'): 32, ('B', 'F3'): 34, ('B', 'F4'): 35, ('B', 'F5'): 38, ('B', 'F6'): 40,
    ('C', 'F1'): 40, ('C', 'F2'): 46, ('C', 'F3'): 42, ('C', 'F4'): 37, ('C', 'F5'): 40, ('C', 'F6'): 50
}


model.product_cost_in_factory = pe.Param (model.product, model.factory, initialize = product_costs_dict)



$Q_j$: capcaity of factory $j$ [unit] <br>

In [63]:

#product capacity
capacities_dict = {
    "F1": 550,
    "F2": 700,
    "F3": 1100,
    "F4": 350,
    "F5": 400,
    "F6": 450
}

model.capacity_factory = pe.Param(model.factory, initialize = capacities_dict)


$SP_i$: selling price of product $i$ [€]

In [64]:
product_selling_price_dict = {
    'A' : 60,
    'B' : 82.5,
    'C' : 108
}
model.product_selling_price = pe.Param(model.product, initialize = product_selling_price_dict)


$SOLD_i$: contract of product $i$ [unit]


In [65]:
sold_products_dict = {
    'A' : 700,
    'B' : 500,
    'C' : 600
}
model.sold_products = pe.Param(model.product, initialize = sold_products_dict)

#### Variables

$q_{ij}$: quantity of product $i$ in factory $j$ [unit]

In [66]:
model.product_quantity_per_factory = pe.Var(model.product,model.factory, within = pe.NonNegativeReals)

#### Objective Function

max $\sum_{ij}(SP_i *q_{ij} ) - \sum_{ij}(C_{ij} *q_{ij} )$

In [67]:
def obj_rule(model):
    return sum(((model.product_selling_price[i] * model.product_quantity_per_factory[i,j]) - (model.product_cost_in_factory[i,j]*model.product_quantity_per_factory[i,j]))
                for i in model.product 
                for j in model.factory )

model.cost = pe.Objective(rule = obj_rule, sense = pe.maximize)


#### Constraints

Constraint #1, maximum capacity of factory $j$ [unit] 

$\sum_i q_{ij} \le Q_j \quad \forall j$

In [68]:
model.capacity_constraint = pe.ConstraintList()
for j in model.factory:
    model.capacity_constraint.add(
        sum(model.product_quantity_per_factory[i, j] for i in model.product) <= model.capacity_factory[j]
    )

Constraint #2, minimum sold product $i$ via contract [unit] 

$\sum_j q_{i,j} \;\geq\; SOLD_i \quad \forall i$

In [69]:
model.contracts_constraint = pe.ConstraintList()
for i in model.product:
    model.contracts_constraint.add(
        sum(model.product_quantity_per_factory[i, j] for j in model.factory) >= model.sold_products[i]
    )

In [70]:
solver = pe.SolverFactory('gurobi')
results = solver.solve(model, tee=True)

# Mostrar estado de la optimización
print("Solver Status:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)

Read LP format model from file C:\Users\beatr\AppData\Local\Temp\tmpelhd01me.pyomo.lp
Reading time = 0.00 seconds
x1: 9 rows, 18 columns, 36 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 18 columns and 36 nonzeros
Model fingerprint: 0x5bbc731c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 1e+03]
Presolve time: 0.00s
Presolved: 9 rows, 18 columns, 36 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.6200000e+32   1.800000e+31   8.620000e+02      0s
      10    2.0520000e+05   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.052000000e+05
Solver S

In [71]:
print("Optimal cost:", pe.value(model.cost))

Optimal cost: 205200.0


In [72]:
# Production plan

for i in model.product:
    for j in model.factory:
        qty = pe.value(model.product_quantity_per_factory[i, j])
        if qty > 1e-6:  # mostramos solo valores positivos
            print(f"{qty:.2f} units of product {i} produced in factory {j}")



200.00 units of product A produced in factory F2
50.00 units of product A produced in factory F3
450.00 units of product A produced in factory F6
500.00 units of product B produced in factory F2
550.00 units of product C produced in factory F1
1050.00 units of product C produced in factory F3
350.00 units of product C produced in factory F4
400.00 units of product C produced in factory F5


In [73]:
# Total production per product
for i in model.product:
    total_prod = sum(pe.value(model.product_quantity_per_factory[i, j]) for j in model.factory)
    print(f"Total production of product {i}: {total_prod:.2f} units")

Total production of product A: 700.00 units
Total production of product B: 500.00 units
Total production of product C: 2350.00 units


In [74]:
# Total production per factory
for j in model.factory:
    total_fac = sum(pe.value(model.product_quantity_per_factory[i, j]) for i in model.product)
    print(f"Total production in factory {j}: {total_fac:.2f} units")

Total production in factory F1: 550.00 units
Total production in factory F2: 700.00 units
Total production in factory F3: 1100.00 units
Total production in factory F4: 350.00 units
Total production in factory F5: 400.00 units
Total production in factory F6: 450.00 units


In [75]:
model.pprint()

2 Set Declarations
    factory : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {'F1', 'F2', 'F3', 'F4', 'F5', 'F6'}
    product : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'A', 'B', 'C'}

4 Param Declarations
    capacity_factory : Size=6, Index=factory, Domain=Any, Default=None, Mutable=False
        Key : Value
         F1 :   550
         F2 :   700
         F3 :  1100
         F4 :   350
         F5 :   400
         F6 :   450
    product_cost_in_factory : Size=18, Index=product*factory, Domain=Any, Default=None, Mutable=False
        Key         : Value
        ('A', 'F1') :    25
        ('A', 'F2') :    30
        ('A', 'F3') :    26
        ('A', 'F4') :    34
        ('A', 'F5') :    32
        ('A', 'F6') :    30
        ('B', 'F1') :    30
        ('B', 'F2') :    32
        ('B', 'F3') :    34
        ('B', 'F4') : 

In [76]:
model.display() 

Model unknown

  Variables:
    product_quantity_per_factory : Size=18, Index=product*factory
        Key         : Lower : Value  : Upper : Fixed : Stale : Domain
        ('A', 'F1') :     0 :    0.0 :  None : False : False : NonNegativeReals
        ('A', 'F2') :     0 :  200.0 :  None : False : False : NonNegativeReals
        ('A', 'F3') :     0 :   50.0 :  None : False : False : NonNegativeReals
        ('A', 'F4') :     0 :    0.0 :  None : False : False : NonNegativeReals
        ('A', 'F5') :     0 :    0.0 :  None : False : False : NonNegativeReals
        ('A', 'F6') :     0 :  450.0 :  None : False : False : NonNegativeReals
        ('B', 'F1') :     0 :    0.0 :  None : False : False : NonNegativeReals
        ('B', 'F2') :     0 :  500.0 :  None : False : False : NonNegativeReals
        ('B', 'F3') :     0 :    0.0 :  None : False : False : NonNegativeReals
        ('B', 'F4') :     0 :    0.0 :  None : False : False : NonNegativeReals
        ('B', 'F5') :     0 :    0.0